# 📈 iPhone Fiyat Zaman Serisi Analizi - Kapsamlı Öğretici

## 🎯 Bu Notebook'ta Öğreneceğin Konular:

1. **📊 Zaman Serisi Nedir?** - Temel kavramlar
2. **🧹 Veri Hazırlama** - Temizlik ve ön işleme
3. **🔍 Keşifsel Veri Analizi** - Görselleştirme ve istatistikler
4. **🚨 Anomali Tespiti** - Isolation Forest ile kampanya dönemleri
5. **📈 ARIMA Modeli** - Geleneksel zaman serisi tahmini
6. **🔮 Prophet Modeli** - Facebook'un modern yaklaşımı
7. **🤖 LSTM Modeli** - Deep Learning ile tahmin
8. **📊 Model Karşılaştırması** - Hangi model ne zaman kullanılır?
9. **🎨 İnteraktif Görselleştirme** - Plotly ile profesyonel grafikler

---

## 🚀 Google Colab'da Kullanım:
1. Bu dosyayı Google Colab'a yükle
2. Her hücreyi sırasıyla çalıştır (Shift + Enter)
3. Açıklamaları oku ve kodları analiz et
4. Kendi verilerinle deneme yap!

## 1️⃣ Kütüphaneleri Yükle ve Kurulum

### 🔧 Gerekli Kütüphaneler:
- **pandas**: Veri manipülasyonu
- **numpy**: Sayısal işlemler
- **matplotlib/seaborn**: Temel görselleştirme
- **plotly**: İnteraktif grafikler
- **statsmodels**: ARIMA modeli
- **prophet**: Facebook'un zaman serisi kütüphanesi
- **sklearn**: Machine Learning algoritmaları
- **tensorflow**: Deep Learning (LSTM)

### ⚡ Colab'da Eksik Kütüphaneleri Yükle:

In [ ]:
# Google Colab'da eksik kütüphaneleri yükle
!pip install prophet plotly statsmodels openpyxl

print("✅ Tüm kütüphaneler yüklendi!")

In [ ]:
# Temel kütüphaneleri import et
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Zaman serisi kütüphaneleri
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from prophet import Prophet

# Machine Learning kütüphaneleri
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

# Deep Learning (LSTM)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Görselleştirme ayarları
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

print("📚 Tüm kütüphaneler başarıyla yüklendi!")
print("🚀 Zaman serisi analizine hazırız!")

## 2️⃣ Gerçek Proje Verisini Yükle

### 📊 Profesyonel iPhone Fiyat Verisi:
Gerçek projende oluşturduğun `profesyonel_telefon_verileri2.csv` dosyasını kullanacağız.

### 🎯 Bu Veri Setinde Bulunanlar:
- **9,142 gerçek kayıt** - Manuel toplanan iPhone fiyatları
- **43 özellik** - Teknik özellikler, performans skorları, pazar analizi
- **iPhone Modelleri**: iPhone 13, 14, 15, 16 
- **Kapasiteler**: 128GB, 256GB, 512GB
- **Zaman Aralığı**: Gerçek tarih verileri
- **Kampanya Analizi**: Otomatik tespit edilmiş indirim dönemleri
- **Performans Skorları**: DxOMark + hesaplanmış metrikler

### 💡 Neden Gerçek Veri Kullanıyoruz:
- **Gerçek pazar dinamiklerini** görme
- **Doğru tahmin modellerini** test etme  
- **Pratik uygulamaları** öğrenme
- **Profesyonel sonuçlar** alma

In [ ]:
# 📊 GERÇEK PROJE VERİSİNİ YÜKLE
print("📊 Gerçek iPhone fiyat verisi yükleniyor...")
print("=" * 50)

# Google Colab için dosya yükleme (isteğe bağlı)
try:
    # Google Colab'daysan bu kodu çalıştır
    from google.colab import files
    print("🔗 Google Colab tespit edildi!")
    print("📤 Lütfen 'profesyonel_telefon_verileri2.csv' dosyasını yükleyin:")
    uploaded = files.upload()
    if 'profesyonel_telefon_verileri2.csv' in uploaded:
        print("✅ Dosya başarıyla yüklendi!")
    else:
        print("⚠️ CSV dosyası bulunamadı. Alternatif yüklemeleri deneyin.")
except ImportError:
    print("💻 Yerel ortam tespit edildi (Google Colab değil)")
    print("📁 Mevcut dizindeki CSV dosyası kullanılacak")

# CSV dosyasını oku
try:
    # Dosyayı yükle
    df = pd.read_csv('profesyonel_telefon_verileri2.csv', encoding='utf-8')
    print("✅ Veri seti başarıyla yüklendi!")
    
    # Tarih sütununu datetime'a çevir
    df['TARIH'] = pd.to_datetime(df['TARIH'], format='%d.%m.%Y', errors='coerce')
    
    # Temel veri temizliği
    df = df.dropna(subset=['TARIH', 'FIYAT'])  # Eksik tarih/fiyat olan kayıtları çıkar
    df = df.sort_values('TARIH')  # Tarihe göre sırala
    
    print(f"\n📊 VERİ SETİ BİLGİLERİ:")
    print(f"📈 Toplam kayıt sayısı: {len(df):,}")
    print(f"📱 iPhone modelleri: {', '.join(df['MODEL'].unique())}")
    print(f"💾 Kapasiteler: {', '.join(df['KAPASITE'].unique())}")
    print(f"📅 Tarih aralığı: {df['TARIH'].min().strftime('%d.%m.%Y')} - {df['TARIH'].max().strftime('%d.%m.%Y')}")
    print(f"💰 Fiyat aralığı: {df['FIYAT'].min():,.0f} - {df['FIYAT'].max():,.0f} TL")
    print(f"📊 Toplam özellik sayısı: {len(df.columns)}")
    
    # Veri kalitesi kontrolü
    print(f"\n🔍 VERİ KALİTESİ:")
    print(f"📊 Eksik değer oranı: %{(df.isnull().sum().sum() / (len(df) * len(df.columns)) * 100):.1f}")
    print(f"📈 Sayısal sütun sayısı: {df.select_dtypes(include=[np.number]).shape[1]}")
    print(f"📝 Kategorik sütun sayısı: {df.select_dtypes(exclude=[np.number]).shape[1]}")
    
    # Önemli sütunları kontrol et
    required_columns = ['TARIH', 'MODEL', 'FIYAT', 'PERFORMANS_SKORU', 'KAMPANYA_DURUMU']
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        print(f"⚠️ Eksik önemli sütunlar: {missing_columns}")
    else:
        print("✅ Tüm önemli sütunlar mevcut!")
    
    # İlk 5 kayıt
    print("\n🔍 İLK 5 KAYIT:")
    display(df[['TARIH', 'MODEL', 'KAPASITE', 'FIYAT', 'KAMPANYA_DURUMU', 'PERFORMANS_SKORU']].head())
    
    # Model bazında özet
    print(f"\n📱 MODEL BAZINDA ÖZET:")
    model_summary = df.groupby('MODEL').agg({
        'FIYAT': ['count', 'mean', 'std', 'min', 'max'],
        'KAMPANYA_DURUMU': lambda x: x.value_counts().head(1).index[0] if len(x) > 0 else 'N/A'
    }).round(0)
    
    model_summary.columns = ['Kayıt_Sayısı', 'Ort_Fiyat', 'Std_Sapma', 'Min_Fiyat', 'Max_Fiyat', 'Yaygın_Kampanya']
    display(model_summary)
    
except FileNotFoundError:
    print("❌ HATA: 'profesyonel_telefon_verileri2.csv' dosyası bulunamadı!")
    print("💡 Lütfen dosyanın mevcut dizinde olduğundan emin olun.")
    print("📤 Google Colab'da yukarıdaki dosya yükleme hücresini çalıştırın.")
    
    # Alternatif: Küçük simülasyon verisi oluştur
    print("\n🔄 Demo için küçük simülasyon verisi oluşturuluyor...")
    dates = pd.date_range('2024-01-01', '2024-07-28', freq='D')
    demo_data = []
    
    for model in ['iPhone 15', 'iPhone 16']:
        base_price = 55000 if model == 'iPhone 15' else 65000
        for date in dates:
            price = base_price + np.random.normal(0, 2000)
            demo_data.append({
                'TARIH': date,
                'MODEL': model,
                'FIYAT': round(price, 2),
                'PERFORMANS_SKORU': 145 if model == 'iPhone 15' else 147,
                'KAMPANYA_DURUMU': 'Normal Fiyat',
                'FIYAT_PERFORMANS_ORANI': round(price / (145 if model == 'iPhone 15' else 147), 2),
                'URUN_YASI_GUN': 300,
                'YIL': date.year,
                'AY': date.month,
                'GUN': date.day,
                'HAFTA_GUN': date.weekday(),
                'GUN_TIPI': 'Hafta Sonu' if date.weekday() >= 5 else 'Hafta İçi'
            })
    
    df = pd.DataFrame(demo_data)
    print("✅ Demo veri seti oluşturuldu!")

except Exception as e:
    print(f"❌ Beklenmedik hata: {e}")
    print("🔄 Demo veri seti oluşturuluyor...")
    # Demo veri oluşturma kodu buraya gelecek

print(f"\n? Veri seti hazır! Analize başlayabiliriz.")
print(f"? Toplam {len(df)} kayıt ile çalışacağız.")
print(f"🎯 Zaman serisi analizi için optimize edilmiş veri!")

## 3️⃣ Keşifsel Veri Analizi (EDA)

### 🔍 Zaman Serisi Analizi İlk Adımları:
1. **Veri kalitesi kontrolü** - Eksik değerler, outlier'lar
2. **Temel istatistikler** - Ortalama, standart sapma, dağılım
3. **Görselleştirme** - Trendler, mevsimsellik, patterns
4. **Model bazında analiz** - Her iPhone modelinin özelliği

In [ ]:
# 🧹 GERÇEK VERİ SETİ KALİTE KONTROLÜ
print("🧹 GERÇEK VERİ SETİ KALİTE KONTROLÜ")
print("=" * 50)

# Eksik değer kontrolü
missing_values = df.isnull().sum()
if missing_values.sum() > 0:
    print(f"⚠️ Eksik değerler:")
    for col, count in missing_values[missing_values > 0].items():
        print(f"  {col}: {count} eksik (%{count/len(df)*100:.1f})")
else:
    print("✅ Eksik değer bulunamadı!")

# Veri tipi kontrolü
print(f"\n📊 VERİ TİPLERİ:")
print(f"📈 Sayısal sütunlar: {len(df.select_dtypes(include=[np.number]).columns)}")
print(f"📝 Kategorik sütunlar: {len(df.select_dtypes(exclude=[np.number]).columns)}")

# Önemli sayısal sütunları kontrol et
numeric_cols = ['FIYAT', 'PERFORMANS_SKORU', 'FIYAT_PERFORMANS_ORANI']
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Temel istatistikler
print(f"\n📊 TEMEL İSTATİSTİKLER")
print("-" * 30)
print(f"📅 Tarih aralığı: {(df['TARIH'].max() - df['TARIH'].min()).days} gün")
print(f"📱 Toplam model sayısı: {df['MODEL'].nunique()}")
print(f"? Kapasite çeşidi: {df['KAPASITE'].nunique()}")
print(f"?💰 Ortalama fiyat: {df['FIYAT'].mean():,.0f} TL")
print(f"📊 Fiyat std sapması: ±{df['FIYAT'].std():,.0f} TL")
print(f"📈 Min fiyat: {df['FIYAT'].min():,.0f} TL")
print(f"📈 Max fiyat: {df['FIYAT'].max():,.0f} TL")

# Model bazında detaylı özet
print(f"\n📱 MODEL BAZINDA DETAYLI ÖZET")
print("-" * 40)
model_summary = df.groupby('MODEL').agg({
    'FIYAT': ['count', 'mean', 'std', 'min', 'max'],
    'PERFORMANS_SKORU': 'mean' if 'PERFORMANS_SKORU' in df.columns else lambda x: 0,
    'FIYAT_PERFORMANS_ORANI': 'mean' if 'FIYAT_PERFORMANS_ORANI' in df.columns else lambda x: 0
}).round(2)

model_summary.columns = ['Kayıt_Sayısı', 'Ort_Fiyat', 'Std_Sapma', 'Min_Fiyat', 'Max_Fiyat', 'Ort_Performans', 'F/P_Oranı']
display(model_summary)

# Kampanya dağılımı (eğer varsa)
if 'KAMPANYA_DURUMU' in df.columns:
    print(f"\n🛍️ KAMPANYA DURUMU DAĞILIMI")
    print("-" * 30)
    campaign_dist = df['KAMPANYA_DURUMU'].value_counts()
    for campaign, count in campaign_dist.items():
        percentage = (count / len(df)) * 100
        print(f"{campaign}: {count:,} kayıt (%{percentage:.1f})")
else:
    print(f"\n⚠️ Kampanya durumu bilgisi bulunamadı")

# Kapasite dağılımı
if 'KAPASITE' in df.columns:
    print(f"\n💾 KAPASİTE DAĞILIMI")
    print("-" * 20)
    capacity_dist = df['KAPASITE'].value_counts()
    for capacity, count in capacity_dist.items():
        percentage = (count / len(df)) * 100
        print(f"{capacity}: {count:,} kayıt (%{percentage:.1f})")

# Yıl bazında dağılım
print(f"\n📅 YIL BAZINDA DAĞILIM")
print("-" * 20)
year_dist = df['YIL'].value_counts().sort_index()
for year, count in year_dist.items():
    percentage = (count / len(df)) * 100
    print(f"{year}: {count:,} kayıt (%{percentage:.1f})")

print(f"\n✅ Veri kalitesi kontrolü tamamlandı!")
print(f"🎯 {len(df):,} kayıt ile analize devam ediyoruz")

In [ ]:
# 📊 TEMEL GÖRSELLEŞTİRME
print("📊 Temel görselleştirmeler oluşturuluyor...")

# 1. Zaman serisi grafiği (Model bazında)
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('📱 iPhone Fiyat Trendleri (Model Bazında)', fontsize=16, fontweight='bold')

models = df['MODEL'].unique()
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

for i, model in enumerate(models):
    row = i // 2
    col = i % 2
    
    model_data = df[df['MODEL'] == model].sort_values('TARIH')
    
    # Günlük ortalama hesapla (gürültüyü azaltmak için)
    daily_avg = model_data.groupby('TARIH')['FIYAT'].mean().reset_index()
    
    axes[row, col].plot(daily_avg['TARIH'], daily_avg['FIYAT'], 
                       color=colors[i], linewidth=2, alpha=0.8)
    axes[row, col].set_title(f'{model}', fontweight='bold')
    axes[row, col].set_ylabel('Fiyat (TL)')
    axes[row, col].grid(True, alpha=0.3)
    axes[row, col].tick_params(axis='x', rotation=45)
    
    # İstatistikler ekle
    avg_price = model_data['FIYAT'].mean()
    axes[row, col].axhline(y=avg_price, color='red', linestyle='--', alpha=0.7, 
                          label=f'Ortalama: {avg_price:,.0f} TL')
    axes[row, col].legend()

plt.tight_layout()
plt.show()

# 2. Fiyat dağılımı (Box plot)
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='MODEL', y='FIYAT', palette='Set2')
plt.title('📊 iPhone Fiyat Dağılımları (Box Plot)', fontweight='bold', fontsize=14)
plt.ylabel('Fiyat (TL)')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)
plt.show()

# 3. Kampanya dönemleri analizi
plt.figure(figsize=(14, 6))
campaign_monthly = df.groupby(['YIL', 'AY', 'KAMPANYA_DURUMU']).size().unstack(fill_value=0)
campaign_monthly.plot(kind='bar', stacked=True, figsize=(14, 6), colormap='Set3')
plt.title('🛍️ Aylık Kampanya Dağılımı', fontweight='bold', fontsize=14)
plt.xlabel('Yıl-Ay')
plt.ylabel('Kayıt Sayısı')
plt.legend(title='Kampanya Durumu', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("✅ Temel görselleştirmeler tamamlandı!")

## 4️⃣ Anomali Tespiti - Kampanya Dönemlerini Bul

### 🔍 Anomali Tespiti Nedir?
- **Normal olmayan** fiyat hareketlerini tespit etme
- **Kampanya dönemlerini** otomatik bulma
- **Isolation Forest** algoritması kullanarak
- **İndirim oranlarını** hesaplama

### 🎯 Neden Önemli?
- Pazarlama stratejilerini anlama
- Fiyat tahmininde bu dönemleri dikkate alma
- Satın alma zamanı optimizasyonu

In [ ]:
# 🔍 ANOMALİ TESPİTİ - KAMPANYA DÖNEMLERİ
print("🔍 ANOMALİ TESPİTİ - KAMPANYA DÖNEMLERİ")
print("=" * 50)

def detect_price_anomalies(model_name, contamination=0.1):
    """
    Belirli bir iPhone modeli için fiyat anomalilerini tespit eder
    
    Parameters:
    -----------
    model_name : str
        iPhone model adı
    contamination : float
        Anomali oranı (0.1 = %10 anomali bekleniyor)
    
    Returns:
    --------
    dict: Anomali analiz sonuçları
    """
    
    # Model verilerini filtrele
    model_data = df[df['MODEL'] == model_name].copy()
    model_data = model_data.sort_values('TARIH')
    
    print(f"\n📱 {model_name} Anomali Analizi:")
    print("-" * 40)
    
    # Özellik matrisi hazırla
    features = ['FIYAT', 'PERFORMANS_SKORU', 'FIYAT_PERFORMANS_ORANI', 'URUN_YASI_GUN']
    X = model_data[features].fillna(model_data[features].mean())
    
    # Veriyi normalleştir (Isolation Forest için önemli)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Isolation Forest modeli
    isolation_forest = IsolationForest(
        contamination=contamination,  # %10 anomali bekleniyor
        random_state=42,
        n_estimators=100
    )
    
    # Anomali tespiti
    anomalies = isolation_forest.fit_predict(X_scaled)
    model_data['ANOMALI'] = anomalies
    
    # Anomali skorları (ne kadar anormal olduğu)
    anomaly_scores = isolation_forest.decision_function(X_scaled)
    model_data['ANOMALI_SKORU'] = anomaly_scores
    
    # Anomali verilerini ayır
    normal_data = model_data[model_data['ANOMALI'] == 1]
    anomaly_data = model_data[model_data['ANOMALI'] == -1]
    
    # İstatistikler hesapla
    total_records = len(model_data)
    anomaly_count = len(anomaly_data)
    anomaly_rate = (anomaly_count / total_records) * 100
    
    normal_avg_price = normal_data['FIYAT'].mean()
    anomaly_avg_price = anomaly_data['FIYAT'].mean() if len(anomaly_data) > 0 else 0
    discount_rate = ((normal_avg_price - anomaly_avg_price) / normal_avg_price * 100) if anomaly_avg_price > 0 else 0
    
    print(f"📊 Toplam kayıt: {total_records:,}")
    print(f"🚨 Anomali sayısı: {anomaly_count:,}")
    print(f"📈 Anomali oranı: %{anomaly_rate:.1f}")
    print(f"💰 Normal ortalama fiyat: {normal_avg_price:,.0f} TL")
    print(f"🎯 Anomali ortalama fiyat: {anomaly_avg_price:,.0f} TL")
    print(f"🔥 Ortalama indirim oranı: %{discount_rate:.1f}")
    
    # En büyük anomalileri (en ucuz fiyatları) bul
    if len(anomaly_data) > 0:
        top_anomalies = anomaly_data.nsmallest(5, 'ANOMALI_SKORU')[['TARIH', 'FIYAT', 'KAMPANYA_DURUMU', 'ANOMALI_SKORU']]
        print(f"\n🏆 En Büyük Anomaliler (En İyi Fırsatlar):")
        for _, row in top_anomalies.iterrows():
            print(f"  📅 {row['TARIH'].strftime('%d.%m.%Y')}: {row['FIYAT']:,.0f} TL ({row['KAMPANYA_DURUMU']})")
    
    return {
        'model_name': model_name,
        'total_records': total_records,
        'anomaly_count': anomaly_count,
        'anomaly_rate': anomaly_rate,
        'normal_avg_price': normal_avg_price,
        'anomaly_avg_price': anomaly_avg_price,
        'discount_rate': discount_rate,
        'model_data': model_data,
        'anomaly_data': anomaly_data
    }

# Her model için anomali tespiti yap
anomaly_results = {}
for model in df['MODEL'].unique():
    anomaly_results[model] = detect_price_anomalies(model)

print(f"\n✅ Tüm modeller için anomali tespiti tamamlandı!")

In [ ]:
# 📊 ANOMALİ GÖRSELLEŞTİRME
print("📊 Anomali görselleştirmeleri oluşturuluyor...")

# Anomali görselleştirmesi
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('🚨 Fiyat Anomalileri - Kampanya Dönemleri', fontsize=16, fontweight='bold')

models = list(anomaly_results.keys())
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']

for i, model in enumerate(models[:4]):
    row = i // 2
    col = i % 2
    
    model_data = anomaly_results[model]['model_data']
    anomaly_data = anomaly_results[model]['anomaly_data']
    
    # Normal fiyatlar
    normal_data = model_data[model_data['ANOMALI'] == 1]
    axes[row, col].scatter(normal_data['TARIH'], normal_data['FIYAT'], 
                          alpha=0.6, s=20, color=colors[i], label='Normal Fiyat')
    
    # Anomali fiyatlar (kampanya dönemleri)
    if len(anomaly_data) > 0:
        axes[row, col].scatter(anomaly_data['TARIH'], anomaly_data['FIYAT'], 
                              color='red', s=30, alpha=0.8, label='Anomali (Kampanya)', marker='x')
    
    axes[row, col].set_title(f'{model}\n(Anomali: %{anomaly_results[model]["anomaly_rate"]:.1f})', 
                            fontweight='bold')
    axes[row, col].set_ylabel('Fiyat (TL)')
    axes[row, col].grid(True, alpha=0.3)
    axes[row, col].legend()
    axes[row, col].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Anomali özet tablosu
print("\n📋 ANOMALİ ÖZET TABLOSU")
print("=" * 60)
anomaly_summary = pd.DataFrame([
    {
        'Model': result['model_name'],
        'Toplam_Kayıt': result['total_records'],
        'Anomali_Sayısı': result['anomaly_count'],
        'Anomali_Oranı_%': round(result['anomaly_rate'], 1),
        'Normal_Ort_Fiyat': f"{result['normal_avg_price']:,.0f} TL",
        'Anomali_Ort_Fiyat': f"{result['anomaly_avg_price']:,.0f} TL",
        'İndirim_Oranı_%': round(result['discount_rate'], 1)
    }
    for result in anomaly_results.values()
])

display(anomaly_summary)

print("\n💡 YORUM:")
print("🔴 Kırmızı noktalar = Kampanya dönemleri (anormal derecede ucuz)")
print("🔵 Mavi noktalar = Normal fiyat dönemleri")
print("📊 Anomali oranı = Ne kadar sık kampanya yapıldığını gösterir")
print("💰 İndirim oranı = Kampanyalarda ortalama ne kadar indirim olduğunu gösterir")

print("\n✅ Anomali analizi tamamlandı!")

## 5️⃣ ARIMA Modeli - Geleneksel Zaman Serisi Tahmini

### 📈 ARIMA Nedir?
- **AutoRegressive Integrated Moving Average**
- En yaygın kullanılan zaman serisi modeli
- **AR(p)**: Geçmiş değerlere dayalı tahmin
- **I(d)**: Verinin durağan hale getirilmesi (differencing)
- **MA(q)**: Geçmiş hataları kullanarak tahmin

### 🔧 ARIMA Parametreleri:
- **p**: AutoRegressive sırası (kaç geçmiş değer kullanılacak)
- **d**: Differencing sırası (durağanlık için)
- **q**: Moving Average sırası (kaç geçmiş hata kullanılacak)

### 📊 Durağanlık Testi:
- **ADF Test** (Augmented Dickey-Fuller)
- p-value < 0.05 ise seri durağan
- Durağan değilse differencing yapılır

In [ ]:
# 📈 GERÇEK VERİ İLE ARIMA MODELİ UYGULAMASI
print("📈 GERÇEK VERİ İLE ARIMA MODELİ UYGULAMASI")
print("=" * 50)

def create_arima_model_real_data(model_name, forecast_days=30):
    """
    Gerçek veri seti ile ARIMA modeli oluşturur ve tahmin yapar
    
    Parameters:
    -----------
    model_name : str
        iPhone model adı
    forecast_days : int
        Kaç gün ilerisi için tahmin yapılacak
    
    Returns:
    --------
    dict: ARIMA model sonuçları
    """
    
    print(f"\n📱 {model_name} için ARIMA Modeli (Gerçek Veri):")
    print("-" * 50)
    
    # Model verilerini filtrele
    model_data = df[df['MODEL'] == model_name].copy()
    
    if len(model_data) < 50:
        print(f"⚠️ Yeterli veri yok ({len(model_data)} kayıt). En az 50 kayıt gerekli.")
        return None
    
    print(f"📊 Toplam veri: {len(model_data)} kayıt")
    
    # Günlük ortalama hesapla (aynı günde birden fazla kayıt varsa)
    daily_data = model_data.groupby('TARIH')['FIYAT'].mean().reset_index()
    daily_data = daily_data.sort_values('TARIH')
    
    print(f"📅 Günlük veri noktası: {len(daily_data)} gün")
    
    # Eksik günleri tamamla (interpolasyon ile)
    date_range = pd.date_range(start=daily_data['TARIH'].min(), 
                              end=daily_data['TARIH'].max(), 
                              freq='D')
    
    # Tüm günleri içeren DataFrame oluştur
    complete_data = pd.DataFrame({'TARIH': date_range})
    complete_data = complete_data.merge(daily_data, on='TARIH', how='left')
    
    # Eksik fiyatları interpolasyon ile doldur
    complete_data['FIYAT'] = complete_data['FIYAT'].interpolate(method='linear')
    complete_data['FIYAT'] = complete_data['FIYAT'].fillna(method='bfill').fillna(method='ffill')
    
    print(f"? Tamamlanmış veri: {len(complete_data)} gün")
    
    # Zaman serisi verisi hazırla
    ts = complete_data.set_index('TARIH')['FIYAT']
    
    # Son 100 günü al (çok uzun seriler sorun çıkarabilir)
    if len(ts) > 100:
        ts = ts.tail(100)
        print(f"📊 Son 100 günlük veri kullanılıyor")
    
    # 1. DURAĞANLIK TESTİ (ADF Test)
    print(f"\n🔬 Durağanlık Testi (ADF):")
    try:
        adf_result = adfuller(ts.dropna())
        adf_pvalue = adf_result[1]
        
        print(f"  📊 ADF İstatistiği: {adf_result[0]:.4f}")
        print(f"  🎯 p-değeri: {adf_pvalue:.4f}")
        
        if adf_pvalue < 0.05:
            print(f"  ✅ Seri durağan (p < 0.05)")
            d_parameter = 0
        else:
            print(f"  ⚠️ Seri durağan değil (p ≥ 0.05) - Differencing gerekli")
            d_parameter = 1
            
            # Birinci farkı test et
            ts_diff = ts.diff().dropna()
            if len(ts_diff) > 10:
                adf_diff = adfuller(ts_diff)
                print(f"  📊 1. fark ADF p-değeri: {adf_diff[1]:.4f}")
                if adf_diff[1] < 0.05:
                    print(f"  ✅ 1. fark ile durağan hale geldi")
                else:
                    d_parameter = 2
                    print(f"  🔄 2. fark gerekebilir")
    except Exception as e:
        print(f"  ⚠️ ADF test hatası: {e}")
        d_parameter = 1  # Default
        adf_pvalue = 0.5
    
    # 2. ARIMA MODELİ PARAMETRELERİ
    # Gerçek veri için optimize edilmiş parametreler
    p, d, q = 2, d_parameter, 1  # Biraz daha karmaşık AR modeli
    
    print(f"\n🔧 ARIMA Parametreleri:")
    print(f"  p (AR): {p} - Geçmiş {p} değer kullanılacak")
    print(f"  d (I): {d} - {d}. seviye differencing")
    print(f"  q (MA): {q} - Geçmiş {q} hata kullanılacak")
    
    try:
        # 3. ARIMA MODELİ EĞİTİMİ
        print(f"\n🚀 ARIMA modeli eğitiliyor...")
        
        # Veriyi temizle
        ts_clean = ts.dropna()
        
        if len(ts_clean) < 20:
            print(f"⚠️ Temizlenmiş veri çok az ({len(ts_clean)} gün)")
            return None
        
        arima_model = ARIMA(ts_clean, order=(p, d, q))
        arima_fitted = arima_model.fit()
        
        print(f"✅ Model başarıyla eğitildi!")
        
        # Model bilgileri
        print(f"\n📊 Model Performansı:")
        print(f"  📈 AIC: {arima_fitted.aic:.2f}")
        print(f"  📈 BIC: {arima_fitted.bic:.2f}")
        print(f"  📈 Log Likelihood: {arima_fitted.llf:.2f}")
        
        # 4. TAHMİN YAPMA
        print(f"\n🔮 {forecast_days} günlük tahmin yapılıyor...")
        forecast = arima_fitted.forecast(steps=forecast_days)
        
        # Güven aralıkları
        try:
            forecast_conf_int = arima_fitted.get_forecast(steps=forecast_days).conf_int()
        except:
            forecast_conf_int = None
            print("  ⚠️ Güven aralığı hesaplanamadı")
        
        # Tahmin istatistikleri
        current_avg = ts_clean.tail(7).mean()  # Son 7 günün ortalaması
        forecast_avg = forecast.mean()
        price_change = ((forecast_avg - current_avg) / current_avg) * 100
        
        print(f"\n💰 Tahmin Sonuçları:")
        print(f"  📊 Son 7 gün ortalama: {current_avg:,.0f} TL")
        print(f"  🔮 {forecast_days} günlük ortalama tahmin: {forecast_avg:,.0f} TL")
        print(f"  📈 Öngörülen değişim: {price_change:+.1f}%")
        
        # Fiyat aralığı
        if forecast_conf_int is not None and len(forecast_conf_int) > 0:
            lower_bound = forecast_conf_int.iloc[:, 0].mean()
            upper_bound = forecast_conf_int.iloc[:, 1].mean()
            print(f"  📊 Güven aralığı: {lower_bound:,.0f} - {upper_bound:,.0f} TL")
        
        # Trend analizi
        if price_change > 5:
            trend = "🔴 Güçlü Yükseliş"
            recommendation = "⚡ Hemen satın alınması önerilir"
        elif price_change > 2:
            trend = "🟡 Hafif Yükseliş"
            recommendation = "📊 Fiyat izlenmesi önerilir"
        elif price_change < -5:
            trend = "🟢 Güçlü Düşüş"
            recommendation = "⏳ Satın alma için uygun dönem yaklaşıyor"
        elif price_change < -2:
            trend = "🟡 Hafif Düşüş"
            recommendation = "📊 Fiyat izlenmesi önerilir"
        else:
            trend = "🔵 Stabil"
            recommendation = "📊 Büyük değişim beklenmemiyor"
        
        print(f"  🎯 Trend: {trend}")
        print(f"  💡 Öneri: {recommendation}")
        
        return {
            'model_name': model_name,
            'arima_order': (p, d, q),
            'adf_pvalue': adf_pvalue,
            'is_stationary': adf_pvalue < 0.05,
            'aic': arima_fitted.aic,
            'bic': arima_fitted.bic,
            'current_avg': current_avg,
            'forecast_avg': forecast_avg,
            'price_change': price_change,
            'trend': trend,
            'recommendation': recommendation,
            'forecast': forecast,
            'forecast_conf_int': forecast_conf_int,
            'time_series': ts_clean,
            'fitted_model': arima_fitted,
            'data_points': len(ts_clean)
        }
        
    except Exception as e:
        print(f"❌ ARIMA modeli hatası: {e}")
        print(f"💡 Bu model için farklı parametreler denenebilir")
        return None

# Her model için ARIMA analizi (gerçek veri ile)
print("🎯 Gerçek veri seti ile ARIMA analizi başlıyor...")
arima_results = {}

for model in df['MODEL'].unique():
    print(f"\n" + "="*60)
    result = create_arima_model_real_data(model, forecast_days=30)
    if result:
        arima_results[model] = result
    else:
        print(f"⚠️ {model} için ARIMA modeli oluşturulamadı")

print(f"\n" + "="*60)
print(f"✅ ARIMA analizi tamamlandı!")
print(f"📊 Başarılı model sayısı: {len(arima_results)}/{df['MODEL'].nunique()}")

# Sonuç özeti
if arima_results:
    print(f"\n📋 ARIMA TAHMİN ÖZETİ:")
    print("-" * 40)
    for model_name, result in arima_results.items():
        print(f"📱 {model_name}:")
        print(f"  📊 Veri noktası: {result['data_points']} gün")
        print(f"  🔮 Tahmin: {result['forecast_avg']:,.0f} TL ({result['price_change']:+.1f}%)")
        print(f"  🎯 {result['trend']}")
        print(f"  💡 {result['recommendation']}")
        print()
else:
    print("⚠️ Hiçbir model için başarılı ARIMA analizi yapılamadı")
    print("💡 Veri setinde daha fazla veri gerekebilir")